In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# ruff: noqa: E402
# find the root of the project
import os
from pathlib import Path
import sys
import polars as pl
import dotenv


ROOT = Path(os.getcwd()).parent
while not ROOT.joinpath(".git").exists():
    ROOT = ROOT.parent

# add the root to the python path
sys.path.append(str(ROOT))


dotenv.load_dotenv(ROOT.joinpath(".env"))

from src.utils import check_gpu_available

GPU = check_gpu_available()
print(f"GPU available: {GPU}")

GPU available: True


## Load the Network


In [3]:
import geopandas as gpd
from src.geometry import RoadNetwork


mainline_net = RoadNetwork(
    lane_gdf=gpd.read_file(ROOT / "data/mainline_lanes.geojson"),
    keep_lanes=["EBL1", "WBL1"],
    step_size=0.01,
)

full_net = RoadNetwork(
    lane_gdf=gpd.read_file(ROOT / "data/mainline_lanes.geojson"),
    keep_lanes=None,
    step_size=0.01,
)

LANE_WIDTH = 3.55
LANE_NUM = 2

## Read in the Trajectories


In [4]:
from datetime import timedelta
import polars as pl
from src.radar import CalibratedRadar
from src.pipelines.open_file import prep_df


USE_FRONT = False


radar_obj = CalibratedRadar(
    radar_location_path=ROOT / "configuration" / "march_calibrated.yaml",
)

In [5]:
# _df = pl.scan_parquet(
#     # Path(os.environ.get("RAW_DATA_DIR")).joinpath("*.parquet")
#     Path("/home/max/Development/ua-traffic-data/tmp/all_working.parquet")
#     # "/Users/max/Development/DOE-Project/ua-traffic-data/tmp/all_working_processed.parquet"
# )

# _df.with_columns(
#     pl.col("epoch_time").dt.replace_time_zone("UTC").dt.convert_time_zone("US/Central")
# ).select(
#     pl.col("epoch_time").min().alias("min"), pl.col("epoch_time").max().alias("max")
# ).collect()

In [6]:
# radar_df = (
#     pl.scan_parquet(
#         Path("/home/max/Development/ua-traffic-data/tmp/all_working.parquet")
#     )
#     .with_columns(
#         pl.col("epoch_time").dt.replace_time_zone("UTC").dt.round("100ms"),
#     )
#     .pipe(prep_df, f=radar_obj)
#     .collect(streaming=True)
# )

In [7]:
# (
#     radar_df.group_by("object_id")
#     .agg(
#         (
#             pl.col("utm_x").diff() ** 2
#             + pl.col("utm_y").diff() ** 2
#         )
#         .sqrt()
#         .sum()
#         .alias("distance")
#     )
#     .select(pl.col("distance").mean())
# )

In [8]:
radar_df = (
    pl.scan_parquet(
        Path("/home/max/Development/ua-traffic-data/tmp/all_working.parquet")
    )
    .with_columns(
        pl.col("epoch_time").dt.replace_time_zone("UTC").dt.round("100ms"),
    )
    .pipe(prep_df, f=radar_obj)
    .collect(streaming=True)
    .pipe(
        full_net.map_to_lane,
        dist_upper_bound=(LANE_WIDTH / 2),
        utm_x_col="utm_x",
        utm_y_col="utm_y",
    )
    .rename(
        {
            "name": "lane",
            "angle": "heading_lane",
        }
    )
)

function: create_object_id took: 0.0001201629638671875 seconds
function: filter_short_trajectories took: 0.0003654956817626953 seconds
function: clip_trajectory_end took: 0.0001983642578125 seconds
function: resample took: 0.00021147727966308594 seconds
function: set_timezone took: 3.9577484130859375e-05 seconds
function: add_cst_timezone took: 2.9325485229492188e-05 seconds
function: add_heading took: 2.5033950805664062e-05 seconds
function: rotate_radars took: 0.0004913806915283203 seconds
function: update_origin took: 0.0004572868347167969 seconds


In [9]:
radar_df = radar_df.with_columns(
    pl.col("epoch_time").dt.replace_time_zone("UTC").dt.convert_time_zone("US/Central")
    - pl.duration(seconds=2.9)
).filter(pl.col("epoch_time").dt.hour() == 7)

In [25]:
res_df = (
    radar_df.filter(pl.col("lane").is_in(["EBL1", "EBL2", "WBL1", "WBL2"]))
    .sort(["epoch_time", "s"])
    .with_columns(pl.col("s").shift(-1).over(["lane", "epoch_time"]).alias("leader_s"))
    .select(
        pl.col("object_id").alias("vehicle_id"),
        "lane",
        "epoch_time",
        "utm_x",
        "utm_y",
        (pl.col("leader_s") - pl.col("s")).alias("gap"),
        pl.col("f32_velocityInDir_mps").alias("speed"),
        ((pl.col('epoch_time') - pl.col('epoch_time').min()).dt.total_milliseconds() / 1000).alias('vehicle_time')
    )
    .sort('vehicle_id', "epoch_time")
)

In [26]:
from scipy.signal import butter, filtfilt, freqz, sosfiltfilt
import matplotlib.pyplot as plt


follower_id = 7446


def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    sos = butter(order, normal_cutoff, btype="low", analog=False, output="sos")
    return sos


sos = butter_lowpass(0.25, 1 / 0.1, order=5)


def butter_lowpass_filter(
    data,
):
    # normalize data
    # max_val = np.max(data)
    # data = data / max_val
    # b, a = butter_lowpass(cutoff, fs, order=order)
    try:
        x = sosfiltfilt(
            sos,
            data,
        )

        if len(x) < len(data):
            return data
        return x
    except:
        return data

    return x


def butter_lowpass_filter_plot(
    df: pl.DataFrame, col: str, vehicle_col: str
) -> pl.DataFrame:
    return df.with_columns(
        pl.col(col)
        .map_elements(
            # lambda x: pl.Series(
            #     name=f"{col}_lowpass",
            #     values=(
            #         # if x.is_not_null().all()
            #         # else x.to_numpy()
            #     ),
            #     dtype=pl.Float64,
            # ),
            lambda x: pl.Series(
                values=butter_lowpass_filter(
                    x.to_numpy(),
                ),
                dtype=float,
            ),
            # return_dtype=pl.Float64,
            # strategy='threading'
        )
        .over(vehicle_col)
        .alias(f"{col}_lowpass")
    )


small_df = res_df.pipe(butter_lowpass_filter_plot, "speed", "vehicle_id").with_columns(
    (pl.col("speed_lowpass").diff() / pl.col("vehicle_time").diff())
    .backward_fill(1)
    .over("vehicle_id")
    .alias("lowpass_accel")
)

In [27]:
small_df

vehicle_id,lane,epoch_time,utm_x,utm_y,gap,speed,vehicle_time,speed_lowpass,lowpass_accel
u64,str,"datetime[ms, US/Central]",f64,f64,f64,f32,f64,f64,f64
2158106745633146,"""WBL1""",2023-03-13 07:44:38.800 CDT,442763.342945,3.6775e6,0.32,1.738027,2678.8,1.632069,2.90375
2158106745633146,"""WBL1""",2023-03-13 07:44:38.900 CDT,442763.199271,3.6775e6,1.030002,1.738027,2678.9,1.922444,2.90375
2158106745633146,"""WBL1""",2023-03-13 07:44:39 CDT,442762.617868,3.6775e6,0.980001,2.1094,2679.0,2.212871,2.904268
2158106745633146,"""WBL1""",2023-03-13 07:44:39.100 CDT,442762.109699,3.6775e6,1.040002,2.428074,2679.1,2.502461,2.895897
2158106745633146,"""WBL1""",2023-03-13 07:44:39.200 CDT,442761.407387,3.6775e6,0.550001,2.879037,2679.2,2.79041,2.879494
2158106745633146,"""WBL1""",2023-03-13 07:44:39.300 CDT,442760.971141,3.6775e6,0.530001,3.03756,2679.3,3.07601,2.855997
2158106745633146,"""WBL1""",2023-03-13 07:44:39.400 CDT,442760.203987,3.6775e6,0.900001,3.475389,2679.4,3.35865,2.826402
2158106745633146,"""WBL1""",2023-03-13 07:44:39.500 CDT,442759.288912,3.6775e6,0.950001,3.945798,2679.5,3.637824,2.791737
2158106745633146,"""WBL1""",2023-03-13 07:44:39.600 CDT,442758.6812,3.6775e6,1.120002,4.175154,2679.6,3.913128,2.753044
